In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neural_network import MLPClassifier

import os
print(os.listdir("../input"))

['bert-submission-dataset', 'cnn-and-rnn-to-submit']


In [2]:
print(os.listdir("../input/bert-submission-dataset"))

['bert_submission_cv.csv', 'bert_submission (1).csv']


In [3]:
val_rnn_submission = pd.read_csv("../input/cnn-and-rnn-to-submit/val_rnn_submission.csv")
val_cnn_submission = pd.read_csv("../input/cnn-and-rnn-to-submit/val_conv_submission.csv")
val_bert_submission = pd.read_csv("../input/bert-submission-dataset/bert_submission_cv.csv")

In [4]:
val_rnn_submission.shape, val_cnn_submission.shape, val_bert_submission.shape

((80858, 3), (80858, 3), (80858, 4))

In [5]:
val_cnn_submission.head()

,cv_id,is_duplicate,val_id
0,NaN,0.504915,112020
1,NaN,0.659953,372698
2,NaN,0.000773,241993
3,NaN,0.864283,170576
4,NaN,0.138700,131381


In [6]:
val_bert_submission.rename(columns={"is_duplicate": "bert_preds"}, inplace = True)
val_rnn_submission.rename(columns={"is_duplicate": "rnn_preds"}, inplace = True)
val_cnn_submission.rename(columns={"is_duplicate": "cnn_preds"}, inplace = True)

In [7]:
cv_dataset = val_bert_submission[['bert_preds', 'cv_id', 'cv_true']]
cv_dataset['rnn_preds'] = val_rnn_submission['rnn_preds']
cv_dataset['cnn_preds'] = val_cnn_submission['cnn_preds']

In [8]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial').fit(cv_dataset[['bert_preds','cnn_preds','rnn_preds']], cv_dataset['cv_true'])

In [9]:
clf.coef_

array([[2.71530652, 0.97091618, 0.31614588]])

In [10]:
bert_weightage = clf.coef_[0,0]
rnn_weightage = clf.coef_[0,2]
cnn_weightage = clf.coef_[0,1]

In [11]:
rnn_predictions = pd.read_csv("../input/cnn-and-rnn-to-submit/rnn_submission.csv")
conv_predictions = pd.read_csv("../input/cnn-and-rnn-to-submit/conv_submission.csv")
bert_predictions = pd.read_csv("../input/bert-submission-dataset/bert_submission (1).csv")

In [12]:
combined_csv = rnn_predictions.copy()
combined_csv.head()

,test_id,is_duplicate
0,0,0.019953
1,1,0.107283
2,2,0.507044
3,3,0.098097
4,4,0.440422


In [13]:
combined_csv['is_duplicate'] = bert_predictions.loc[:,'is_duplicate']*bert_weightage + conv_predictions.loc[:,'is_duplicate'] * cnn_weightage + rnn_predictions.loc[:,'is_duplicate']*rnn_weightage
combined_csv['is_duplicate'] = combined_csv['is_duplicate']/np.sum(clf.coef_)
combined_csv[combined_csv['is_duplicate']<=0.001].is_duplicate = 0
combined_csv[combined_csv['is_duplicate']>=0.99].is_duplicate = 1
combined_csv.to_csv('combined.csv', index=False)                 

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
